In [1]:
"""
Bicicletas recorridos
Clima PRCP TAVG
2020-2021
acquisition clean prepare upload
"""
import os
import sys

import urllib.request
# from zipfile import ZipFile

import pandas as pd
import numpy as np

import random

# Mover esto a exploring
# import matplotlib.pyplot as plt
# import seaborn as sns
# import matplotlib as mpl

# mpl.rcParams.update( mpl.rcParamsDefault ) 

In [3]:
bicicletas = {
    "recorridos" : {
        "2020" : {
            "url" : 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte/bicicletas-publicas/recorridos-realizados-2020.zip',
            "zip_file" : "./datasets/recorridos-realizados-2020.zip",
            "df" : pd.DataFrame(),
        },
        "2021" : {
            "url" : 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte/bicicletas-publicas/recorridos-realizados-2021.zip',   
            "zip_file" : "./datasets/recorridos-realizados-2021.zip",
            "df" : pd.DataFrame(),
        },
    },
    "recorridos_df" : pd.DataFrame(),
    "estaciones": {
        "salidas_df" : pd.DataFrame(),
        "arribos_df" : pd.DataFrame(),
    }
}

In [4]:
"""
Descargar recorridios bicicletas
"""
def descargar_recoridos(
    recorridos,
    equivalencias_encabezados
):
    for r in recorridos:
        if not os.path.exists( recorridos[r]["zip_file"] ):
            print("Descargando", recorridos[r]["url"])
            urllib.request.urlretrieve(
              recorridos[r]["url"],
              recorridos[r]["zip_file"]
            )
        else:
            print("Ya descargado", recorridos[r]["zip_file"])  
        recorridos[r]["df"] = pd.read_csv(
            recorridos[r]["zip_file"],
            compression = 'zip',
            encoding = 'utf-8', 
        )
#         print( recorridos[r]["df"].info() )
    # ajustar 2021
    recorridos["2021"]["df"] = recorridos["2021"]["df"]
    recorridos["2021"]["df"] = recorridos["2021"]["df"].rename( columns = equivalencias_encabezados )
    recorridos["2021"]["df"]["id"] = recorridos["2021"]["df"]['id'].astype('Int64')
    print('done')
    return recorridos

equivalencias_encabezados = {
        'ID'                                  : 'id',
        'Duración'                            : 'duracion_recorrido',
        'Estado cerrado'                      : 'estado_cerrado',
        'ID de ciclista'                      : 'id_usuario',
        'Tipo de ciclista'                    : 'cilicsta_tipo',
        'Modelo de bicicleta'                 : 'bicicleta_modelo',   

        'Id de estación de inicio'            : 'id_estacion_origen', 
        'Nombre de estación de inicio'        : 'nombre_estacion_origen',
        'Fecha de inicio'                     : 'fecha_origen_recorrido',     
        'Origen de viaje'                     : 'direccion_estacion_origen',  
        # '':'long_estacion_origen',        
        # '':'lat_estacion_origen',
        'Id de estación de fin de viaje'      : 'id_estacion_destino',     
        'Nombre de estación de fin de viaje'  : 'nombre_estacion_destino',
        'Fecha de fin'                        : 'fecha_destino_recorrido',          
        # '':'direccion_estacion_destino',           
        # '':'long_estacion_destino',       
        # '':'lat_estacion_destino',
        # '':'periodo',  
}
bicicletas["recorridos"] = descargar_recoridos(
    bicicletas["recorridos"],
    equivalencias_encabezados
)

Ya descargado ./datasets/recorridos-realizados-2020.zip
Ya descargado ./datasets/recorridos-realizados-2021.zip
done


In [5]:
bicicletas["recorridos"]["2021"]["df"].info()
bicicletas["recorridos"]["2020"]["df"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1328726 entries, 0 to 1328725
Data columns (total 13 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   id                         1328726 non-null  Int64  
 1   estado_cerrado             1328726 non-null  object 
 2   duracion_recorrido         1328726 non-null  int64  
 3   id_estacion_origen         1328726 non-null  int64  
 4   fecha_origen_recorrido     1328726 non-null  object 
 5   nombre_estacion_origen     1328726 non-null  object 
 6   fecha_destino_recorrido    1328726 non-null  object 
 7   id_estacion_destino        1328723 non-null  float64
 8   nombre_estacion_destino    1328723 non-null  object 
 9   id_usuario                 1328726 non-null  int64  
 10  cilicsta_tipo              1328726 non-null  object 
 11  bicicleta_modelo           1328726 non-null  object 
 12  direccion_estacion_origen  1328726 non-null  object 
dtypes: Int64(1),

In [6]:
# Concatenar recorridos 2020 2021
frames = []
for r in bicicletas["recorridos"]:
    frames.append( bicicletas["recorridos"][r]["df"] )
bicicletas["recorridos_df"] = pd.concat( frames )

In [7]:
bicicletas["recorridos_df"] 

,duracion_recorrido,id_estacion_origen,fecha_origen_recorrido,nombre_estacion_origen,fecha_destino_recorrido,id_estacion_destino,nombre_estacion_destino,id_usuario,direccion_estacion_origen,long_estacion_origen,lat_estacion_origen,direccion_estacion_destino,long_estacion_destino,lat_estacion_destino,periodo,id,estado_cerrado,cilicsta_tipo,bicicleta_modelo
0,882,116,2020-12-31 23:54:22,116 - HOSPITAL ALEMÁN,2021-01-01 00:09:04,214.0,142 - Armenia y Gorriti,666202,Beruti & Ecuador,-58.402586,-34.592171,Armenia 1525 y Gorriti,-58.428972,-34.590541,2020.0,<NA>,NaN,NaN,NaN
1,894,116,2020-12-29 12:39:18,116 - HOSPITAL ALEMÁN,2020-12-29 12:54:12,214.0,142 - Armenia y Gorriti,332689,Beruti & Ecuador,-58.402586,-34.592171,Armenia 1525 y Gorriti,-58.428972,-34.590541,2020.0,<NA>,NaN,NaN,NaN
2,1361,116,2020-12-21 21:21:38,116 - HOSPITAL ALEMÁN,2020-12-21 21:44:19,214.0,142 - Armenia y Gorriti,57949,Beruti & Ecuador,-58.402586,-34.592171,Armenia 1525 y Gorriti,-58.428972,-34.590541,2020.0,<NA>,NaN,NaN,NaN
3,1010,116,2020-12-20 21:58:19,116 - HOSPITAL ALEMÁN,2020-12-20 22:15:09,214.0,142 - Armenia y Gorriti,454846,Beruti & Ecuador,-58.402586,-34.592171,Armenia 1525 y Gorriti,-58.428972,-34.590541,2020.0,<NA>,NaN,NaN,NaN
4,1490,116,2020-12-20 13:32:53,116 - HOSPITAL ALEMÁN,2020-12-20 13:57:43,214.0,142 - Armenia y Gorriti,690975,Beruti & Ecuador,-58.402586,-34.592171,Armenia 1525 y Gorriti,-58.428972,-34.590541,2020.0,<NA>,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1328721,690,277,2021-01-01 00:09:43,292 - PLAZA BOLIVIA,2021-01-01 00:21:13,44.0,044 - Ecoparque,62246,CODE,NaN,NaN,NaN,NaN,NaN,NaN,9646396,NORMAL,MEMBER,ICONIC
1328722,1360,79,2021-01-01 00:08:40,079 - AZUCENA VILLAFLOR,2021-01-01 00:31:20,168.0,168 - Estados Unidos,445201,CODE,NaN,NaN,NaN,NaN,NaN,NaN,9646395,NORMAL,MEMBER,ICONIC
1328723,1169,79,2021-01-01 00:08:37,079 - AZUCENA VILLAFLOR,2021-01-01 00:28:06,8.0,008 - Congreso,554162,CODE,NaN,NaN,NaN,NaN,NaN,NaN,9646394,NORMAL,MEMBER,ICONIC
1328724,1031,79,2021-01-01 00:06:50,079 - AZUCENA VILLAFLOR,2021-01-01 00:24:01,75.0,075 - Plaza Primero de Mayo,51005,CODE,NaN,NaN,NaN,NaN,NaN,NaN,9646393,NORMAL,MEMBER,ICONIC


In [8]:
# Fill nan en id y destinos
# bicicletas["recorridos_df"]["id"] = bicicletas["recorridos_df"]["id"].interpolate( method='linear' ) # no funco
bicicletas["recorridos_df"] = bicicletas["recorridos_df"].sort_values( by=['fecha_origen_recorrido'] )
bicicletas["recorridos_df"]["id"] = bicicletas["recorridos_df"]["id"].fillna(0)
bicicletas["recorridos_df"]['nombre_estacion_destino'] = bicicletas["recorridos_df"]['nombre_estacion_destino'].fillna('0 - null')

In [9]:
"""
Separar las columnas nombre de estacion de origen y nombre de estacion de destino 
Tienen la info correcta del id de las estaciones.

Divido la columna Nombre de estacion de fin de viaje porque contiene dos campos 
el Id y el nombre de la estacion de inicio de viaje.
"""

# Use vectoried str.split with expand = True:
bicicletas["recorridos_df"][
    ['id_estacion_incio','nombre_estacion_origen']
] = bicicletas["recorridos_df"]['nombre_estacion_origen'].str.split( ' - ', expand = True )

# (registros Nan ya convertidos a '0 - null')
bicicletas["recorridos_df"][
    ['id_estacion_destino','nombre_estacion_destino']
] = bicicletas["recorridos_df"]['nombre_estacion_destino'].str.split( ' - ', expand = True )
# bicicletas["recorridos_df"]["id_estacion_incio"] = bicicletas["recorridos_df"]["id_estacion_incio"].astype('Int64')

In [13]:
# sacar los leading zeros de id_estacion_destino
bicicletas["recorridos_df"]['id_estacion_destino'] = bicicletas["recorridos_df"]['id_estacion_destino'].str.lstrip('0')
# bicicletas["recorridos_df"]["id_estacion_destino"] = bicicletas["recorridos_df"]["id_estacion_destino"].astype('Int64')

In [14]:
bicicletas["recorridos_df"]

,duracion_recorrido,id_estacion_origen,fecha_origen_recorrido,nombre_estacion_origen,fecha_destino_recorrido,id_estacion_destino,nombre_estacion_destino,id_usuario,direccion_estacion_origen,long_estacion_origen,lat_estacion_origen,direccion_estacion_destino,long_estacion_destino,lat_estacion_destino,periodo,id,estado_cerrado,cilicsta_tipo,bicicleta_modelo,id_estacion_incio
593758,6563,30,2020-01-01 00:02:50,Peña,2020-01-01 01:52:13,30,Peña,506009,Peña & Azcuenaga,-58.397367,-34.590822,Peña & Azcuenaga,-58.397367,-34.590822,2020.0,0,NaN,NaN,NaN,030
687904,1150,206,2020-01-01 00:04:19,Fitz Roy & Gorriti,2020-01-01 00:23:29,123,Armenia,116847,Fitz Roy & Gorriti,-58.437337,-34.584950,Armenia 2315 & Charcas,-58.420869,-34.585138,2020.0,0,NaN,NaN,NaN,159
1026453,383,308,2020-01-01 00:05:23,Solar de la Abadía,2020-01-01 00:11:46,255,BARRANCAS DE BELGRANO,259973,Maure 1819,-58.436750,-34.567633,"Sucre, Antonio Jose De, Mcal. Y Vertiz Virrey Av.",-58.448313,-34.559801,2020.0,0,NaN,NaN,NaN,251
1876322,72,92,2020-01-01 00:05:42,Salcedo,2020-01-01 00:06:54,92,Salcedo,354594,2908 Salcedo & Dean Funes,-58.405336,-34.631646,2908 Salcedo & Dean Funes,-58.405336,-34.631646,2020.0,0,NaN,NaN,NaN,092
1126779,769,41,2020-01-01 00:07:30,Urquiza y Rondeau,2020-01-01 00:20:19,41,Urquiza y Rondeau,222181,"Urquiza, Gral. & Rondeau",-58.407632,-34.636453,"Urquiza, Gral. & Rondeau",-58.407632,-34.636453,2020.0,0,NaN,NaN,NaN,041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,591,212,2021-06-02 16:09:32,Velasco,2021-06-02 16:19:23,158,VILLARROEL,35926,CODE,NaN,NaN,NaN,NaN,NaN,NaN,11204919,NORMAL,MEMBER,ICONIC,129
3,400,117,2021-06-02 16:09:34,HUMBERTO 1°,2021-06-02 16:16:14,147,Constitución,713938,CODE,NaN,NaN,NaN,NaN,NaN,NaN,11204920,NORMAL,MEMBER,ICONIC,117
2,299,232,2021-06-02 16:10:08,Tamborini,2021-06-02 16:15:07,277,Coghlan,690073,CODE,NaN,NaN,NaN,NaN,NaN,NaN,11204928,NORMAL,MEMBER,ICONIC,252
1,406,387,2021-06-02 16:11:01,Donato Alvarez,2021-06-02 16:17:47,349,Plaza Saenz Peña,348428,CODE,NaN,NaN,NaN,NaN,NaN,NaN,11204938,NORMAL,MEMBER,ICONIC,278


In [16]:
# Separo estaciones grupos artificiales (clusters)  
# revisar viajes cortos y misma estacion de salida llegada
# estratos naturales (grupos),

# SALIDAS
bicicletas["estaciones"]["salidas_df"] = bicicletas["recorridos_df"].groupby(
    ['id_estacion_origen','nombre_estacion_origen',]
).agg(
  cantidad_salidas = pd.NamedAgg( column = "id_estacion_origen", aggfunc = "count" ),
    # recorridos_q_vuelven
    # para buscar recorridos que vuelven a la misma estacion
#   cantidad_arribos = pd.NamedAgg( column="id_estacion_destino", aggfunc="count" ), 
  # recorridos_cortos
).reset_index()
bicicletas["estaciones"]["salidas_df"]["nombre_estacion_origen"] = bicicletas["estaciones"]["salidas_df"]["nombre_estacion_origen"].str.upper()

# ARRIBOS
bicicletas["estaciones"]["arribos_df"] = bicicletas["recorridos_df"].groupby(
    ['id_estacion_destino','nombre_estacion_destino',]
).agg(
  cantidad_arribos = pd.NamedAgg( column = "id_estacion_origen", aggfunc = "count" ),
).reset_index()
bicicletas["estaciones"]["arribos_df"]["nombre_estacion_destino"] = bicicletas["estaciones"]["arribos_df"]["nombre_estacion_destino"].str.upper()

In [21]:
bicicletas["estaciones"]["salidas_df"] = bicicletas["estaciones"]["salidas_df"].sort_values( by = ['cantidad_salidas'], ascending = False )
bicicletas["estaciones"]["arribos_df"] = bicicletas["estaciones"]["arribos_df"].sort_values( by = ['cantidad_arribos'], ascending = False ) 

In [23]:
bicicletas["estaciones"]["arribos_df"]

,id_estacion_destino,nombre_estacion_destino,cantidad_arribos
50,160,GODOY CRUZ Y LIBERTADOR,48368
33,14,PACIFICO,44859
139,29,PARQUE CENTENARIO,44650
140,292,PLAZA BOLIVIA,44648
116,255,BARRANCAS DE BELGRANO,40928
...,...,...,...
6,105,15 DE NOVIEMBRE,178
167,36,MAIPÚ,168
194,47,COLEGIO NACIONAL DE BUENOS AIRES,4
0,,NULL,3


In [4]:
bicicletas["estaciones"]["salidas_df"]

NameError: name 'bicicletas' is not defined

In [3]:
# Filtro y orden columnas que quedan
orden_filtro = [
    'id',
    'duracion_recorrido',
#     'estado_cerrado',
    'id_usuario',
#     'cilicsta_tipo',
#     'bicicleta_modelo',
    'id_estacion_origen',
#     'nombre_estacion_origen',       
#     'direccion_estacion_origen',
    'fecha_origen_recorrido',       
#     'long_estacion_origen',     
#     'lat_estacion_origen',
    'id_estacion_destino',         
    'nombre_estacion_destino',    
#     'direccion_estacion_destino',
    'fecha_destino_recorrido',      
#     'long_estacion_destino',       
#     'lat_estacion_destino', 
#     'periodo',                       
]
bicicletas["recorridos_df"] = bicicletas["recorridos_df"][orden_filtro]

NameError: name 'bicicletas' is not defined

In [2]:
bicicletas["recorridos_df"]

NameError: name 'bicicletas' is not defined

### Procesar fecha

Parsear object datetime a dia, mes, año

In [ ]:
formato = '%Y%m%d %H:%M:%S'
bicicletas["recorridos_df"]['start_datetime'] = pd.to_datetime(
    bicicletas["recorridos_df"]['fecha_origen_recorrido'], 
    format = formato
)
# bicicletas["recorridos_df"]['start_date'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%Y-%m-%d')
bicicletas["recorridos_df"]['anio'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%Y')
bicicletas["recorridos_df"]['nmes'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%m')
# bicicletas["recorridos_df"]['mes']  = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%b').str.lower()
bicicletas["recorridos_df"]['ndia'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%j')
# bicicletas["recorridos_df"]['dia']  = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%a').str.lower()
bicicletas["recorridos_df"]['wdia'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%w')
bicicletas["recorridos_df"]['hora'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%H')
bicicletas["recorridos_df"]['mins'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%M')
bicicletas["recorridos_df"]

In [ ]:
bicicletas["recorridos_df"]

In [ ]:
bicicletas["recorridos_clima_df"] = pd.merge(
    left = bicicletas["recorridos_df"],
    right = weather_data_df,
    left_on = 'start_datetime',
    right_on = 'DATE'
)

In [ ]:
# Exportar CSV

In [14]:
compression_opts = dict(
    method ='zip',
    archive_name='./datasets/recorridos_2020-2021.csv'
)  

bicicletas["recorridos_df"].to_csv(
    './datasets/recorridos_2020-2021.zip',
    index = False,
    compression = compression_opts
)

# Load to GoogleDrive

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/drive')

In [ ]:
# Fetch weather from drive
id = '14BpBfjQ3IrfeK8CajCWrEdfPg7S0eqc8'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('weather-data-ba.csv') 
weather_data_df = pd.read_csv('weather-data-ba.csv')
weather_data_df.head()
weather_data_df = weather_data_df.filter(
    ['DATE','PRCP','TAVG']
)
weather_data_df.isnull().sum( axis = 0 )
weather_data_df['PRCP'] = weather_data_df['PRCP'].replace( np.nan, 0 )
weather_data_df.head()

In [ ]:
bicicletas["recorridos_clima_df"].to_csv(
    '/drive/My Drive/Datasets-bicing/recorridos-clima-2020-2021.csv'
)

In [ ]:
bicicletas["recorridos_clima_df"].head()

## Bajar el ultimo
Bajar el dataset limpio desde drive

In [ ]:
from google.colab import files
files.download('recorridos_2020-2021.zip')

In [ ]:
bicicletas["recorridos_df"].to_csv(
    '/drive/My Drive/Datasets-bicing/recorridos_2020-2021.csv'
)

In [ ]:
id = '17liAPD1lyj2wT-Hcbh4ed9-Cn4peoyIq'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('recorridos_2020-2021.csv') 
bicicletas["recorridos_df"] = pd.read_csv('recorridos_2020-2021.csv')
bicicletas["recorridos_df"].head()

In [ ]:
# bicicletas["estaciones"]["salidas_df"] = bicicletas["recorridos_df"].groupby(
#     ['id_estacion_origen','nombre_estacion_origen',]
# ).agg(
#   cantidad_salidas = pd.NamedAgg( column = "id_estacion_origen", aggfunc = "count" ),
    
#     # recorridos_q_vuelven
#     # para buscar recorridos que vuelven a la misma estacion
# #   cantidad_arribos = pd.NamedAgg( column="id_estacion_destino", aggfunc="count" ), 
    
#   # recorridos_cortos
    
# #   max_height = pd.NamedAgg( column="height", aggfunc="max"),
# #   PROMEDIO_DURACION = pd.NamedAgg( column="DURACION", aggfunc = np.mean ),
# ).reset_index()